In [1]:
import tensorflow as tf

In [2]:
file_url = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'

In [3]:
zip_dir = tf.keras.utils.get_file('cats_and_dogs.zip', origin=file_url, extract=True)

In [4]:
import pathlib

In [5]:
path = pathlib.Path(zip_dir).parent / 'cats_and_dogs_filtered'

In [6]:
train_dir = path / 'train'
validation_dir = path / 'validation'

In [7]:
train_cats_dir = train_dir / 'cats'
train_dogs_dir = train_dir /'dogs'
validation_cats_dir = validation_dir / 'cats'
validation_dogs_dir = validation_dir / 'dogs'

In [8]:
import os

In [9]:
total_train = len(os.listdir(train_cats_dir)) + len(os.listdir(train_dogs_dir))
total_val = len(os.listdir(validation_cats_dir)) + len(os.listdir(validation_dogs_dir))

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [11]:
train_image_generator = ImageDataGenerator(rescale=1./255)
validation_image_generator = ImageDataGenerator(rescale=1./255)

In [12]:
batch_size=32
img_height = 224
img_width = 224

In [13]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(img_height, img_width),
                                                           class_mode='binary')

Found 2000 images belonging to 2 classes.


In [14]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=validation_dir,
                                                              target_size=(img_height, img_width),
                                                              class_mode='binary')

Found 1000 images belonging to 2 classes.


In [15]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

In [16]:
np.random.seed(8)
tf.random.set_seed(8)

In [17]:
from tensorflow.keras.applications import NASNetMobile

In [18]:
base_model = NASNetMobile(include_top=False, input_shape=(img_height, img_width , 3), weights='imagenet')

In [19]:
base_model.trainable =  False

In [20]:
base_model.summary()

Model: "NASNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
stem_conv1 (Conv2D)             (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn1 (BatchNormalization)   (None, 111, 111, 32) 128         stem_conv1[0][0]                 
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           stem_bn1[0][0]                   
_____________________________________________________________________________________________

In [21]:
model = tf.keras.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(500, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [22]:
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(0.001), metrics=['accuracy'])

In [24]:
model.fit(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=5,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size
)

Epoch 1/5
62/62 [==============================] - 19s 303ms/step - loss: 1.3225 - accuracy: 0.9609 - val_loss: 0.3378 - val_accuracy: 0.9889
Epoch 2/5
62/62 [==============================] - 14s 230ms/step - loss: 0.1606 - accuracy: 0.9924 - val_loss: 0.3385 - val_accuracy: 0.9859
Epoch 3/5
62/62 [==============================] - 14s 230ms/step - loss: 0.0281 - accuracy: 0.9985 - val_loss: 0.3276 - val_accuracy: 0.9849
Epoch 4/5
62/62 [==============================] - 14s 229ms/step - loss: 0.0323 - accuracy: 0.9959 - val_loss: 0.3352 - val_accuracy: 0.9859
Epoch 5/5
62/62 [==============================] - 14s 229ms/step - loss: 0.0086 - accuracy: 0.9990 - val_loss: 0.4509 - val_accuracy: 0.9839
